**Carregamento e Preparação dos Dados**

In [24]:
import tensorflow as tf
from tensorflow.keras import layers, utils
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import os
import re
import shutil

In [15]:
# --- CONFIGURAÇÃO ---
dataset_dir = os.path.join('data', 'aclImdb')
train_dir = os.path.join(dataset_dir, 'train')
test_dir = os.path.join(dataset_dir, 'test')

BATCH_SIZE = 32
VALIDATION_SPLIT = 0.2
SEED = 42

# Remove a pasta 'unsup' para evitar problemas!
unsup_dir = os.path.join(train_dir, 'unsup')
if os.path.exists(unsup_dir):
    shutil.rmtree(unsup_dir)
    print(f"🧹 Pasta intrusa removida: {unsup_dir}")
else:
    print("✨ A pasta 'unsup' não existe mais. Tudo limpo.")

# --- 1. CARREGAR TREINO ---
print("\nCarregando dados de TREINO...")
train_dataset = utils.text_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset='training',
    seed=SEED
)

# --- 2. CARREGAR VALIDAÇÃO ---
print("Carregando dados de VALIDAÇÃO...")
validation_dataset = utils.text_dataset_from_directory(
    train_dir,
    batch_size=BATCH_SIZE,
    validation_split=VALIDATION_SPLIT,
    subset='validation',
    seed=SEED
)

# --- 3. CARREGAR TESTE ---
print("Carregando dados de TESTE...")
test_dataset = utils.text_dataset_from_directory(
    test_dir,
    batch_size=BATCH_SIZE
)

# --- VERIFICAÇÃO FINAL ---
print("\n✅ Classes finais:", train_dataset.class_names)
# deve aparecer apenas: ['neg', 'pos']

✨ A pasta 'unsup' não existe mais. Tudo limpo.

Carregando dados de TREINO...
Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Carregando dados de VALIDAÇÃO...
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
Carregando dados de TESTE...
Found 25000 files belonging to 2 classes.

✅ Classes finais: ['neg', 'pos']


In [16]:
# Salvando o nome das classes
class_names = train_dataset.class_names
print(f"Nomes das classes: {class_names}") # Deve imprimir ['neg', 'pos']

# Aplicando otimizaão
# A partir daqui, train_dataset muda de tipo e fica mais rápido
train_dataset = train_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
validation_dataset = validation_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
test_dataset = test_dataset.cache().prefetch(buffer_size=tf.data.AUTOTUNE)

print("Pipeline de dados otimizado com sucesso!")

Nomes das classes: ['neg', 'pos']
Pipeline de dados otimizado com sucesso!


**Pré-processamento do Texto**

O coração do pré-processamento será a camada TextVectorization. Ela executa três tarefas essenciais:

1- Padronização: Limpa o texto (remove pontuação, converte para minúsculas, etc.).

2- Tokenização: Divide o texto em palavras individuais (tokens).

3- Vetorização: Converte cada token em um número inteiro.

Esta camada será treinada com nossos dados e depois integrada diretamente ao modelo, garantindo que o pré-processamento seja consistente durante o treino, avaliação e inferência.

In [18]:
# 1. Definir parâmetros para a vetorização
VOCAB_SIZE = 10000  # Manter as 10.000 palavras mais frequentes
SEQUENCE_LENGTH = 250 # Padronizar todas as críticas para terem 250 palavras

# 2. Criar a camada de vetorização de texto
# A camada já lida com a conversão para minúsculas e remoção de pontuação por padrão
vectorize_layer = layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH
)

# 3. Treinar a camada de vetorização nos dados de treino
# É crucial usar apenas os dados de treino para construir o vocabulário,
# evitando vazamento de dados (data leakage) do conjunto de teste.
# Mapeamos o dataset para extrair apenas o texto, descartando os rótulos.
train_text = train_dataset.map(lambda text, label: text)
vectorize_layer.adapt(train_text)

# --- Verificação do Processo ---
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

# Exemplo de como um lote de texto é convertido em números
text_batch, label_batch = next(iter(train_dataset))
first_review, first_label = text_batch[0], label_batch[0]
print("--- Exemplo de Vetorização ---")
print("Crítica Original:", first_review.numpy().decode('utf-8'))
# Usar a variável 'class_names' (salva antes de aplicar cache/prefetch) em vez de acessar attribute no dataset
print("Rótulo:", class_names[int(first_label.numpy())])
print("\nCrítica Vetorizada:", vectorize_layer(tf.constant([first_review.numpy()])).numpy()[0, :15])
print("Palavras correspondentes:", vectorize_layer.get_vocabulary()[12:20]) # Exemplo de vocabulário

--- Exemplo de Vetorização ---
Crítica Original: "Pandemonium" is a horror movie spoof that comes off more stupid than funny. Believe me when I tell you, I love comedies. Especially comedy spoofs. "Airplane", "The Naked Gun" trilogy, "Blazing Saddles", "High Anxiety", and "Spaceballs" are some of my favorite comedies that spoof a particular genre. "Pandemonium" is not up there with those films. Most of the scenes in this movie had me sitting there in stunned silence because the movie wasn't all that funny. There are a few laughs in the film, but when you watch a comedy, you expect to laugh a lot more than a few times and that's all this film has going for it. Geez, "Scream" had more laughs than this film and that was more of a horror film. How bizarre is that?<br /><br />*1/2 (out of four)
Rótulo: neg

Crítica Vetorizada: [   1    7    4  194   18 2941   12  256  127   51  384   71  167  257
   70]
Palavras correspondentes: [np.str_('that'), np.str_('br'), np.str_('was'), np.str_('as')

**Construção do Modelo RNN (com LSTM)**

Com o pré-processamento definido, podemos construir a arquitetura da nossa rede neural.

1- Camada de Embedding: Transforma os números inteiros (tokens) em vetores densos de tamanho fixo (embedding_dim). Isso permite que o modelo aprenda o significado e as relações entre as palavras.

2- Camada Bidirecional (LSTM): O núcleo da nossa RNN. A LSTM (Long Short-Term Memory) é uma variante de RNN eficaz para capturar dependências de longo prazo no texto. Usar Bidirectional permite que a rede processe a sequência de texto tanto da esquerda para a direita quanto da direita para a esquerda, capturando um contexto mais rico.

3- Camadas Densas: As camadas finais que atuam como um classificador sobre as features extraídas pela LSTM. A última camada usa a ativação sigmoid, pois a saída é binária (Positivo/Negativo).

In [19]:
# Dimensão do Embedding (representação vetorial de cada palavra)
EMBEDDING_DIM = 16

model = tf.keras.Sequential([
    # 1. A camada de entrada que espera strings de texto
    layers.Input(shape=(1,), dtype=tf.string),
    
    # 2. A camada de pré-processamento que definimos
    vectorize_layer,
    
    # 3. Camada de Embedding: Mapeia o vocabulário para vetores densos
    layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM),
    
    # 4. Camada LSTM Bidirecional para aprender com as sequências
    # O uso de Bidirectional é uma prática comum e eficaz em NLP
    layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
    layers.Bidirectional(layers.LSTM(16)), # Adicionando uma segunda camada LSTM para mais profundidade
    
    # 5. Camadas de classificação final
    layers.Dense(32, activation='relu'),
    layers.Dropout(0.5), # Dropout para regularização e combate ao overfitting
    layers.Dense(1, activation='sigmoid') # Saída binária (0 ou 1)
])

**Compilação e Treinamento do Modelo**

Iremos configurar o otimizador, a função de perda e as métricas para monitorar o treinamento.

*loss='binary_crossentropy': Ideal para problemas de classificação binária.

*optimizer='adam': Um otimizador robusto e amplamente utilizado.

*metrics=['accuracy']: Para monitorar a acurácia durante o treinamento.

In [20]:
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Visualizar a arquitetura do modelo
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ text_vectorization_1            │ (None, 250)            │             0 │
│ (TextVectorization)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 250, 16)        │       160,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 250, 64)        │        12,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional_1 (Bidirectional) │ (None, 32)             │        10,368 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 184,001 (718.75 KB)

 Trainable params: 184,001 (718.75 KB)

 Non-trainable params: 0 (0.00 B)

In [26]:
# Treinamento: alimentamos o modelo com os dados de treino e validação.
print("Iniciando o treinamento com Early Stopping...")

# 1. Definir o Early Stopping
# patience=3: O treinamento para se a val_loss não diminuir por 3 épocas consecutivas.
# restore_best_weights=True: Restaura os pesos do modelo da época onde a val_loss foi a menor.
early_stopping_callback = EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)

# Definimos o EPOCHS para um número alto, confiando no callback para parar no ponto certo
EPOCHS = 100 

history = model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=EPOCHS,
    # A CORREÇÃO ESTÁ AQUI: Passando o callback para o Keras
    callbacks=[early_stopping_callback] 
)

print("Treinamento concluído.")

Iniciando o treinamento com Early Stopping...
Epoch 1/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 107s 172ms/step - accuracy: 0.9933 - loss: 0.0317 - val_accuracy: 0.8524 - val_loss: 0.7774
Epoch 2/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 114s 182ms/step - accuracy: 0.9937 - loss: 0.0279 - val_accuracy: 0.8532 - val_loss: 0.8144
Epoch 3/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 119s 191ms/step - accuracy: 0.9948 - loss: 0.0253 - val_accuracy: 0.8490 - val_loss: 0.8477
Epoch 4/100
625/625 ━━━━━━━━━━━━━━━━━━━━ 120s 191ms/step - accuracy: 0.9962 - loss: 0.0204 - val_accuracy: 0.8456 - val_loss: 0.9570
Treinamento concluído.
